# Recommending Locations for an Italian Restaurant in Fort Worth

In [1]:
# package imports
import requests
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from bs4 import BeautifulSoup

## Data Collection

### Demographics

In [2]:
# API Endpoints for Fort Worth and National Data
fort_worth_url = "https://api.censusreporter.org/1.0/data/show/latest?geo_ids=16000US4827000&table_ids=B01003,B19013,B01002"
us_url = "https://api.censusreporter.org/1.0/data/show/latest?geo_ids=01000US&table_ids=B01003,B19013,B01002"

# Fetch data
fort_worth_data = requests.get(fort_worth_url).json()
us_data = requests.get(us_url).json()

# Extract key statistics
fort_worth_geo = fort_worth_data["data"]["16000US4827000"]
us_geo = us_data["data"]["01000US"]

# Land area values (Census QuickFacts)
fort_worth_land_area = 342.2  # Fort Worth land area (sq. miles)
us_land_area = 3531905  # US total land area (sq. miles)

# Calculate population density
fort_worth_population = fort_worth_geo["B01003"]["estimate"]["B01003001"]
us_population = us_geo["B01003"]["estimate"]["B01003001"]

fort_worth_density = fort_worth_population / fort_worth_land_area
us_density = us_population / us_land_area

# Convert to DataFrame
df = pd.DataFrame({
    "Metric": ["Median Household Income", "Median Age", "Population Density (per sq. mile)"],
    "Fort Worth": [
        fort_worth_geo["B19013"]["estimate"]["B19013001"],  # Fort Worth Median Income
        fort_worth_geo["B01002"]["estimate"]["B01002001"],  # Fort Worth Median Age
        round(fort_worth_density, 2)  # Fort Worth Population Density
    ],
    "National Average": [
        us_geo["B19013"]["estimate"]["B19013001"],  # US Median Income
        us_geo["B01002"]["estimate"]["B01002001"],  # US Median Age
        round(us_density, 2)  # US Population Density
    ]
})

# Display comparison table
print(df)

# Explain the comparison
print("\n{0} is {1:.2f} times than the US medium household income.".format("Fort Worth", fort_worth_geo["B19013"]["estimate"]["B19013001"] / us_geo["B19013"]["estimate"]["B19013001"]))
print("{0} is {1:.2f} younger than the US median age.".format("Fort Worth", abs(fort_worth_geo["B01002"]["estimate"]["B01002001"] - us_geo["B01002"]["estimate"]["B01002001"])))
print("{0} is {1:.2f} times denser than the US average.".format("Fort Worth", fort_worth_density / us_density))


                              Metric  Fort Worth  National Average
0            Median Household Income    77082.00          77719.00
1                         Median Age       33.70             39.20
2  Population Density (per sq. mile)     2854.86             94.83

Fort Worth is 0.99 times than the US medium household income.
Fort Worth is 5.50 younger than the US median age.
Fort Worth is 30.11 times denser than the US average.


From the Census demographic data, we can determine the fort worth is a urban city with a younger population. The household income in this city is similar to the national average. This will be the target demographic when exploring possible locations for a resturant. 

### Competitive Landscape

In [3]:
# Specify the path to the CSV file
csv_path = './data/zomato/zomato.csv'  # Adjust the path if the CSV file is named differently

# Read the CSV file into a pandas DataFrame with specified encoding
df = pd.read_csv(csv_path, encoding='ISO-8859-1')

# Show the first few rows of the dataframe
df.head()

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,Botswana Pula(P),Yes,No,No,No,3,4.5,Dark Green,Excellent,591
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",...,Botswana Pula(P),Yes,No,No,No,4,4.4,Green,Very Good,270
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"Japanese, Sushi",...,Botswana Pula(P),No,No,No,No,4,4.9,Dark Green,Excellent,365
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"Japanese, Korean",...,Botswana Pula(P),Yes,No,No,No,4,4.8,Dark Green,Excellent,229


In [4]:
# info() method to display the data types of each column
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9551 entries, 0 to 9550
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Restaurant ID         9551 non-null   int64  
 1   Restaurant Name       9551 non-null   object 
 2   Country Code          9551 non-null   int64  
 3   City                  9551 non-null   object 
 4   Address               9551 non-null   object 
 5   Locality              9551 non-null   object 
 6   Locality Verbose      9551 non-null   object 
 7   Longitude             9551 non-null   float64
 8   Latitude              9551 non-null   float64
 9   Cuisines              9542 non-null   object 
 10  Average Cost for two  9551 non-null   int64  
 11  Currency              9551 non-null   object 
 12  Has Table booking     9551 non-null   object 
 13  Has Online delivery   9551 non-null   object 
 14  Is delivering now     9551 non-null   object 
 15  Switch to order menu 

In [5]:
# Clean the data

# Drop unnecessary columns
df = df.drop(['Locality', 'Latitude', 'Longitude', 'Rating color', 'Locality Verbose', 'Currency', 'Is delivering now', 'Switch to order menu', 'Address'], axis=1)

#Drop NaN values
df = df.dropna()

# Display the first few rows of the cleaned dataframe
df.head()

,Restaurant ID,Restaurant Name,Country Code,City,Cuisines,Average Cost for two,Has Table booking,Has Online delivery,Price range,Aggregate rating,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"French, Japanese, Desserts",1100,Yes,No,3,4.8,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,Japanese,1200,Yes,No,3,4.5,Excellent,591
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Seafood, Asian, Filipino, Indian",4000,Yes,No,4,4.4,Very Good,270
3,6318506,Ooma,162,Mandaluyong City,"Japanese, Sushi",1500,No,No,4,4.9,Excellent,365
4,6314302,Sambo Kojin,162,Mandaluyong City,"Japanese, Korean",1500,Yes,No,4,4.8,Excellent,229


In [6]:
# Filter Data

#Filter to find US cities
df = df[df['Country Code'] == 216]

#Filter to find Italian restaurants
cuisine = 'Italian'
df = df[df['Cuisines'].str.contains(cuisine, case=False)]

#Sort by rating
df = df.sort_values(by='Aggregate rating', ascending=False)

# Remove rows with no votes
df = df[df['Votes'] > 0]

# Display the first few rows of the filtered dataframe
df.head()

,Restaurant ID,Restaurant Name,Country Code,City,Cuisines,Average Cost for two,Has Table booking,Has Online delivery,Price range,Aggregate rating,Rating text,Votes
507,17095098,Mazzaro's Italian Market,216,Tampa Bay,"Italian, Deli",10,No,No,1,4.9,Excellent,1424
512,17100307,Ella's Americana Folk Art Cafe,216,Tampa Bay,"International, Italian, Southern",40,No,No,3,4.8,Excellent,1715
146,17303642,Flatbread Neapolitan Pizzeria,216,Boise,"Italian, Pizza",25,No,No,2,4.6,Excellent,615
375,17580160,New Yorker Deli & Pizzeria,216,Pensacola,"Italian, Pizza, Sandwich",10,No,No,1,4.6,Excellent,792
255,17258036,Centro,216,Des Moines,"Italian, Pizza",40,No,No,3,4.5,Excellent,1109


In [7]:
# Average rating of Italian restaurants in the US
average_rating = df['Aggregate rating'].mean()

#Average cost for two at Italian restaurants in the US
average_cost = df['Average Cost for two'].mean()

# Display the average rating and average cost
print("Average Rating for Italian Restaurant in the US:", average_rating)
print("Average Cost for two for for Italian Restaurant in the US:", average_cost)

Average Rating for Italian Restaurant in the US: 4.036842105263157
Average Cost for two for for Italian Restaurant in the US: 28.55263157894737


### Accessibility & Foot Traffic


### Crime & Safety Metrics